# 1. Install OAuth Dependencies

In [ ]:
pip install Authlib Flask-OAuthlib

### OAuth Providers
<table>
    <thead>
        <tr>
            <th>Provider</th>
            <th>OAuth URL</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Google OAuth</td>
            <td><a href="https://developers.google.com/identity/protocols/oauth2" target="_blank">Google OAuth</a></td>
        </tr>
        <tr>
            <td>Facebook OAuth</td>
            <td><a href="https://developers.facebook.com/docs/facebook-login/" target="_blank">Facebook OAuth</a></td>
        </tr>
        <tr>
            <td>GitHub OAuth</td>
            <td><a href="https://docs.github.com/en/developers/apps/building-oauth-apps" target="_blank">GitHub OAuth</a></td>
        </tr>
        <tr>
            <td>Twitter OAuth</td>
            <td><a href="https://developer.twitter.com/en/docs/authentication/oauth-2-0" target="_blank">Twitter OAuth</a></td>
        </tr>
        <tr>
            <td>LinkedIn OAuth</td>
            <td><a href="https://www.linkedin.com/developers/docs/oauth-2-0" target="_blank">LinkedIn OAuth</a></td>
        </tr>
        <tr>
            <td>Microsoft OAuth</td>
            <td><a href="https://learn.microsoft.com/en-us/azure/active-directory/develop/v2-oauth2-auth-code-flow" target="_blank">Microsoft OAuth</a></td>
        </tr>
        <tr>
            <td>GitLab OAuth</td>
            <td><a href="https://docs.gitlab.com/ee/administration/oauth.html" target="_blank">GitLab OAuth</a></td>
        </tr>
        <tr>
            <td>Slack OAuth</td>
            <td><a href="https://api.slack.com/authentication/oauth-v2" target="_blank">Slack OAuth</a></td>
        </tr>
        <tr>
            <td>Spotify OAuth</td>
            <td><a href="https://developer.spotify.com/documentation/general/guides/authorization-guide/" target="_blank">Spotify OAuth</a></td>
        </tr>
        <tr>
            <td>Twitch OAuth</td>
            <td><a href="https://dev.twitch.tv/docs/authentication/getting-tokens-oauth" target="_blank">Twitch OAuth</a></td>
        </tr>
        <tr>
            <td>Yahoo OAuth</td>
            <td><a href="https://developer.yahoo.com/oauth/" target="_blank">Yahoo OAuth</a></td>
        </tr>
        <tr>
            <td>Amazon OAuth</td>
            <td><a href="https://developer.amazon.com/docs/login-with-amazon/overview.html" target="_blank">Amazon OAuth</a></td>
        </tr>
        <tr>
            <td>Reddit OAuth</td>
            <td><a href="https://www.reddit.com/dev/api/" target="_blank">Reddit OAuth</a></td>
        </tr>
        <tr>
            <td>Instagram OAuth</td>
            <td><a href="https://developers.facebook.com/docs/instagram-basic-display-api" target="_blank">Instagram OAuth</a></td>
        </tr>
    </tbody>
</table>

# 2. Google Cloud Console

## 2.1 Enter Google Cloud Console
https://console.cloud.google.com/?inv=1&invt=AbiKNA

## 2.2 Select desired Google Account

## 2.3 Create New Project
- 1. Specify its name only

## 2.4 Select This Project

## 2.5 Go to APIs & Services/Library

## 2.6 Enable Google+ API

## 2.7 Go to APIs & Services/OAuth consent screen
- 1. In  **User Type** select External.
- 2. In **App information** add app name and user support email and developer contact information.
- 3. In **Scopes** save and continue.
- 4. In **Test users** add email for testing.
- 5. In **Summary** back to dashboard.

## 2.8 Go to APIs & Services/Credentials
- 1. Create Credentials.
- 2. Select OAuth client ID.
- 3. Select Application Type: "web application" for flask apps.
- 4. Name it as u like.
- 5. In **Authorized redierct URIs**(when are we redirect after login) add URI,ex: http://localhost:5000/authorize/google

## 2.9 Copy ClientId and Client Secret to the Flask APP

# 3. Register Google OAuth

In [ ]:
from flask import flask
from authlib.integrations.flask_client import OAuth

app = Flask(__name__)
oauth = OAuth(app)

# Store Secret Keys in .env file and load them using os and dotenv

# Register The google OAuth
google = oauth.register(
    name='google',
    client_id=app.config["CLIENT_ID"],
    client_secret=app.config['CLIENT_SECRET'],
    server_metadata_url="https://accounts.google.com/.well-known/openid-configuration"
    # The metadata URI provided by Google for OpenID Connect.
    client_kwargs={'scope':'openid profile email'}
    # Additional arguments for the OAuth client configuration.
    # 'scope' specifies the permissions your app is requesting from the user:
    # - 'openid': Basic authentication and user ID.
    # - 'profile': Access to the user's profile information (e.g., name, picture).
    # - 'email': Access to the user's email address.
)

### Google OAuth Scopes

#### 1. openid
- **Info Provided**: Basic authentication information such as the user's **unique identifier (sub)**.
- **Use Case**: Required for OpenID Connect, which allows you to authenticate users.

#### 2. profile
- **Info Provided**:
    - **Name**: The full name of the user.
    - **Given name**: The user's first name.
    - **Family name**: The user's last name.
    - **Profile URL**: A URL to the user's Google profile.
    - **Profile picture**: The URL to the user's profile picture.
- **Use Case**: To retrieve basic profile information like name and picture.

#### 3. email
- **Info Provided**:
    - **Email address**: The user’s email.
    - **Email verified status**: Whether the email is verified by Google.
- **Use Case**: To access the user's email address.


#### 4. Additional Scopes
- https://www.googleapis.com/auth/drive
- https://www.googleapis.com/auth/drive.file
- https://www.googleapis.com/auth/userinfo.profile
- https://www.googleapis.com/auth/userinfo.email
- https://www.googleapis.com/auth/plus.me
- ...

# 4. Create Routes for the Google Login

In [ ]:
# login for google
@app.route('/login/google')
def login_google():
    try:
        redirect_uri = url_for("authorize_google",_external=True)
        return google.authorize_redirect(redirect_uri)
    
    except Exception as e:
        app.logger.error(f"Error during login: {str(e)}")
        return

In [ ]:
# authorize for google
@app.route("/authorize/google")
def authorize_google():
    token = google.authorize_access_token()
    userinfo_endpoint = google.server_metadata['userinfo_endpoint']
    response = google.get(userinfo_endpoint)
    user_info = response.json()
    
    # Get Necessary Infos about user from Google OAuth
    google_id = user_info['sub']
    username = user_info['name']
    email = user_info['email']
    profilepicture = user_info['picture']
    
    # Rest depends on u (store to db,redirect to dashboard,...)

#### Example Response from google.get(userinfo_endpoint):

In [ ]:
{
  "sub": "1234567890",
  "name": "John Doe",
  "given_name": "John",
  "family_name": "Doe",
  "profile": "https://plus.google.com/1234567890",
  "picture": "https://example.com/photo.jpg",
  "email": "johndoe@example.com",
  "email_verified": true,
  "locale": "en"
}

# 5. Setting up Template

In [ ]:
<body>
    <h1>Login with Google</h1>
    <a href="{{ url_for('login_google') }}">Login with Google</a>
</body>

# Sources
- <a href="https://www.youtube.com/watch?v=wctDfjx4xIw&ab_channel=CodewithJosh">Secure OAuth 2.0 in Flask: Python User Authentication Guide by Code with Josh</a>